In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os

# Connect to the database running in Docker
DATABASE_URL = "postgresql://postgres:postgres@db:5432/elevator_db"
engine = create_engine(DATABASE_URL)

In [2]:
def load_elevator_data():
    query = """
    SELECT 
        ec.call_time,
        ec.called_from_floor,
        ec.destination_floor,
        ec.passenger_count,
        AVG(fo.occupancy_count) as avg_occupancy,
        rp.floor as resting_floor,
        EXTRACT(HOUR FROM ec.call_time) as hour_of_day,
        EXTRACT(DOW FROM ec.call_time) as day_of_week,
        ec.wait_time_seconds
    FROM elevator_call ec
    LEFT JOIN floor_occupancy fo ON 
        fo.building_id = (SELECT building_id FROM elevator WHERE id = ec.elevator_id)
        AND fo.floor = ec.called_from_floor
        AND fo.timestamp <= ec.call_time
        AND fo.timestamp >= ec.call_time - interval '5 minutes'
    LEFT JOIN resting_position rp ON
        rp.elevator_id = ec.elevator_id
        AND rp.start_time <= ec.call_time
        AND (rp.end_time >= ec.call_time OR rp.end_time IS NULL)
    GROUP BY 
        ec.id,
        ec.call_time,
        ec.called_from_floor,
        ec.destination_floor,
        ec.passenger_count,
        rp.floor,
        ec.wait_time_seconds
    ORDER BY ec.call_time
    """
    return pd.read_sql(query, engine)

In [3]:

# Load and display the data
df = load_elevator_data()
print("Data shape:", df.shape)
df.head()

Data shape: (223, 9)


,call_time,called_from_floor,destination_floor,passenger_count,avg_occupancy,resting_floor,hour_of_day,day_of_week,wait_time_seconds
0,2025-01-20 00:31:00,4,1,2,3.166667,2,0.0,1.0,179
1,2025-01-20 00:35:00,1,4,1,1.833333,2,0.0,1.0,98
2,2025-01-20 00:48:00,1,0,2,1.833333,2,0.0,1.0,122
3,2025-01-20 00:51:00,2,0,2,2.333333,2,0.0,1.0,63
4,2025-01-20 00:57:00,4,1,2,2.000000,2,0.0,1.0,163


In [4]:
import torch
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Prepare features
features = ['call_time''called_from_floor', 'destination_floor', 'passenger_count', 
           'avg_occupancy', 'hour_of_day', 'day_of_week']
target = 'resting_floor'

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(df[features])
y = df[target].values

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

In [5]:
class ElevatorModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 5)  # 5 floors (0-4)
        )
    
    def forward(self, x):
        return self.layers(x)